# LLM Unit Tester

In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# setup env

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
claude_Key = os.getenv("ANTHROPIC_API_KEY")

In [3]:
# grab instances of apis

openai = OpenAI()
claude = anthropic.Anthropic()

In [4]:
# define constants

OPENAI_MODEL = "gpt-4o"

In [5]:
# define system message

system_message = "You are an expert Python developer specializing in writing clean, precise, and comprehensive unit tests."
system_message += "Your goal is to create test cases for the user's provided Python code."
system_message += "Respond only with Python code. "

In [16]:
# define user prompt function

def user_prompt(python):
    user_prompt = "Create test cases for the provided Python code. "
    user_prompt += "Respond only with Python code. "
    user_prompt += python
    return user_prompt


In [7]:
# define messages function 

def message_to(python):
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_prompt(python)}
    ]
    return messages


In [9]:
# write file

def write_output(python):
    code = python
    with open("unit_tests.py", "w") as f:
        f.write(code)


In [10]:
# gpt function

def create_tests_gpt(python):
    messages = message_to(python)
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        stream=True
    )

    response = ''
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        response += fragment
        # print(fragment, end='', flush=True)
        yield response
    write_output(response)

In [11]:
# test python code

python_code = '''
def calculate_average(numbers: list[float]) -> float:
    """
    Calculates the average of a list of numbers.

    Args:
        numbers: A list of numbers (integers or floats).

    Returns:
        The average of the numbers.

    Raises:
        ValueError: If the input list is empty.
    """
    if not numbers:
        raise ValueError("Input list cannot be empty")
    
    return sum(numbers) / len(numbers)

'''

In [12]:
print(python_code)


def calculate_average(numbers: list[float]) -> float:
    """
    Calculates the average of a list of numbers.

    Args:
        numbers: A list of numbers (integers or floats).

    Returns:
        The average of the numbers.

    Raises:
        ValueError: If the input list is empty.
    """
    if not numbers:
        raise ValueError("Input list cannot be empty")

    return sum(numbers) / len(numbers)




In [13]:
create_tests_gpt(python_code)

<generator object create_tests_gpt at 0x137172240>

In [17]:
# def build gradio app

with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Raw Python code:", lines=10, value=python_code)
        py_comment = gr.Textbox(label="Commented Python Code:", lines=10)
    with gr.Row():
        convert = gr.Button("Build Tests")

    convert.click(create_tests_gpt, inputs=[python], outputs=[py_comment])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
